In [19]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


In [20]:
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)


Files already downloaded and verified


tensor([[[[3, 9, 1,  ..., 6, 8, 6],
          [1, 0, 8,  ..., 3, 3, 1],
          [5, 5, 2,  ..., 1, 0, 6],
          ...,
          [6, 6, 9,  ..., 9, 6, 0],
          [0, 6, 7,  ..., 8, 4, 1],
          [9, 5, 0,  ..., 0, 7, 6]],

         [[6, 2, 1,  ..., 6, 1, 9],
          [7, 4, 5,  ..., 3, 3, 8],
          [7, 9, 0,  ..., 3, 0, 1],
          ...,
          [9, 4, 1,  ..., 4, 2, 7],
          [4, 4, 3,  ..., 6, 3, 8],
          [0, 9, 7,  ..., 1, 8, 8]],

         [[0, 5, 3,  ..., 2, 0, 4],
          [3, 0, 0,  ..., 9, 9, 1],
          [8, 8, 1,  ..., 2, 0, 4],
          ...,
          [7, 2, 0,  ..., 1, 6, 2],
          [9, 4, 5,  ..., 9, 9, 8],
          [7, 9, 9,  ..., 8, 6, 7]]],


        [[[3, 8, 7,  ..., 8, 4, 5],
          [8, 1, 7,  ..., 4, 3, 5],
          [0, 3, 4,  ..., 0, 0, 5],
          ...,
          [9, 2, 0,  ..., 6, 5, 8],
          [2, 7, 3,  ..., 8, 2, 9],
          [3, 5, 7,  ..., 1, 4, 2]],

         [[5, 1, 9,  ..., 1, 5, 4],
          [3, 8, 6,  ..., 3, 3

In [119]:
A=torch.rand(size=(2,3,32,32))
A

patch_size=8
B,C,H,W = A.shape
A = A.reshape(B, C, H // patch_size, patch_size, W // patch_size, patch_size)
A = A.permute(0, 2, 4, 1, 3, 5)  # [B, H', W', C, p_H, p_W]
A = A.flatten(1, 2)  # [B, H'*W', C, p_H, p_W]
A = A.flatten(3, 4)  # [B, H'*W', C, p_H, p_W]
A = A.flatten(2,3)  # [B, H'*W', C, p_H, p_W]
p=torch.matmul(A,A.permute(0,2,1))
p=torch.nn.functional.softmax(p,dim=2)
print("A")
print(A.shape)
print("P")
print(p.shape)

A = torch.matmul(p,A)


# A=A.reshape(B,(H // patch_size) *(W // patch_size),1,C*patch_size**2)
#
print("After all")
print(A.shape)
# A = A.permute(0, 1, 3, 2)  # [B, H', W', C, p_H, p_W]
# print("after permutation")
# print(A.shape)


A
torch.Size([2, 16, 192])
P
torch.Size([2, 16, 16])
After all
torch.Size([2, 16, 192])


In [70]:
p
torch.sum(p,dim=1)

tensor([[1.0000, 0.9999, 1.0000, 0.9959, 0.9999, 1.0004, 0.9993, 1.0001, 0.9999,
         0.9999, 1.0009, 1.0012, 1.0023, 1.0003, 0.9999, 1.0001],
        [1.0000, 1.0012, 0.9999, 1.0000, 0.9998, 1.0002, 1.0000, 1.0000, 0.9995,
         0.9999, 1.0000, 1.0000, 0.9996, 1.0000, 1.0000, 0.9998]])

In [74]:
# p=A*A.T
p=p.T

p.shape


torch.Size([16, 16, 2])

In [123]:
class MyViT(nn.Module):
    def __init__(self, patch_size=8):
        super(MyViT, self).__init__()
        self.patch_size = patch_size
        self.input_dim = 3*patch_size**2

        self.fc1 = nn.Linear(self.input_dim,    self.input_dim*2)
        self.fc2 = nn.Linear(self.input_dim*2,  self.input_dim)
        self.fc3 = nn.Linear(self.input_dim,    self.input_dim//2)
        self.fc4 = nn.Linear(self.input_dim//2, 10)

    def forward(self, x):

        B, C, H, W = x.shape
        x = x.reshape(B, C, H // patch_size, patch_size, W // patch_size, patch_size)
        x = x.permute(0, 2, 4, 1, 3, 5)  # [B, H', W', C, p_H, p_W]
        x = x.flatten(1, 2)  # [B, H'*W', C, p_H, p_W]
        x = x.flatten(3, 4)  # [B, H'*W', C, p_H*p_W]
        x = x.flatten(2,3)  # [B, H'*W', C*p_H*p_W]
        p = torch.matmul(x,x.permute(0,2,1)) # [B, H'*W', H'*W']

        d_k = torch.tensor(C* H* W//(patch_size**2))
        # d_k=d_k.
        print((d_k))
        p = torch.nn.functional.softmax(p/torch.sqrt(d_k),dim=2)
        x = torch.matmul(p,x) # [B, H'*W', C*p_H*p_W ]
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)
        x=torch.sum(x,dim=1) # [B, C*p_H*p_W//2 ]
        x=self.fc4(x)        # [B, 10]

        output = F.log_softmax(x, dim=1)
        return output


In [124]:
mymodel=MyViT()

In [126]:
result=mymodel(torch.rand(2,3,32,32))

tensor(48)


torch.Size([2, 10])